In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('../awesomebook-master/data/customer.csv')
  hotel_tb = pd.read_csv('../awesomebook-master/data/hotel.csv')
  reserve_tb = pd.read_csv('../awesomebook-master/data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('../awesomebook-master/data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('../awesomebook-master/data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('../awesomebook-master/data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('../awesomebook-master/data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
production_tb = load_production()

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 홀드아웃 검증을 위한 데이터 분할
# 예측 모델의 입력값과 예측 대상의 값을 각각 train_test_split 함수에 설정
# test_size는 검증 데이터의 배율
train_data, test_data, train_target, test_target = \
  train_test_split(production_tb.drop('fault_flg', axis=1),
                   production_tb[['fault_flg']],
                   test_size=0.2)

# train_test_split로 행 이름을 현재의 행번호로 바꿈
train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)
train_target.reset_index(inplace=True, drop=True)
test_target.reset_index(inplace=True, drop=True)

# 대상 행번호 리스트를 생성
row_no_list = list(range(len(train_target)))

# 교차 검증을 위한 데이터 분할
k_fold = KFold(n_splits=4, shuffle=True)

# 교차수 만큼 반복 처리, 병렬처리도 가능한 부분
for train_cv_no, test_cv_no in k_fold.split(row_no_list):

  # 교차 검증에 사용할 학습 데이터 추출
  train_cv = train_data.iloc[train_cv_no, :]

  # 교차 검증에 사용할 검증 데이터 추출
  test_cv = train_data.iloc[test_cv_no, :]

  # train_data와 train_target를 학습 데이터로,
  # test_data와 test_target을 검증 데이터로 기계 학습 모델의 구축, 검증

# 교차 검증의 결과 정리

# train을 학습 데ㅣ터로, private_test를 검증 데이터로 하여 기계 학습 모델의 구축, 검증


In [5]:
monthly_index_tb = load_monthly_index()

# train_window_start에 맨 처음 학습 데이터의 시작 행 번호를 지정
train_window_start = 1
# train_window_end에 맨 처음 학습 데이터의 종류 행 번호를 지정
train_window_end = 24
# horizon에 검증 데이터의 데이터 수를 지정
horizon = 12
# skip에 이동할 데이터 수를 설정
skip = 12

# 연월을 기준으로 데이터 정렬
monthly_index_tb.sort_values(by='year_month')

while True:
  # 검증 데이터의 종료 행 번호를 계산
  test_window_end = train_window_end + horizon

  # 행 번호를 지정하여 원본 데이터에서 학습 데이터를 구함
  # train_window_start를 1로 고정하면 학습 데이터를 늘려나가는 검증으로 변환 가능
  train = monthly_index_tb[train_window_start:train_window_end]

  # 행 번호를 지정하여 원본 데이터에서 검증 데이터를 구함
  test = monthly_index_tb[(train_window_end + 1):test_window_end]

  # 검증 데이터의 종료 행 번호가 원본 데이터의 행 수 이상인지 판정
  if test_window_end >= len(monthly_index_tb.index):
    # 모든 데이터를 처리했으면 종료
    break

  # 데아터를 이동 시킨다
  train_window_start += skip
  train_window_end += skip

# 교차 검증의 결과 정리
